<a href="https://colab.research.google.com/github/KanoknatKr/For-me/blob/main/Austin_bike.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import pandas as pd
import sqlite3
import kagglehub

# ดึงพาธของไฟล์จาก Kaggle โดยไม่ต้องดาวน์โหลด
path = kagglehub.dataset_download("jboysen/austin-bike")

stations_path = f"{path}/austin_bikeshare_stations.csv"
trips_path = f"{path}/austin_bikeshare_trips.csv"

# โหลดข้อมูลเข้า Pandas
df_stations = pd.read_csv(stations_path)
df_trips = pd.read_csv(trips_path)

# สร้าง SQLite Database บนหน่วยความจำ
conn = sqlite3.connect(":memory:")
df_stations.to_sql("bikeshare_stations", conn, index=False, if_exists="replace")
df_trips.to_sql("bikeshare_trips", conn, index=False, if_exists="replace")


649231

# 🔥 ตอบคำถาม 🔥

## 1️⃣ แปลงค่า DayOfWeek เป็นชื่อวัน

In [39]:
dayofweek_query = """
SELECT
    start_time,
    CASE CAST(STRFTIME('%w', start_time) AS INTEGER)
        WHEN 0 THEN 'Sunday'
        WHEN 1 THEN 'Monday'
        WHEN 2 THEN 'Tuesday'
        WHEN 3 THEN 'Wednesday'
        WHEN 4 THEN 'Thursday'
        WHEN 5 THEN 'Friday'
        WHEN 6 THEN 'Saturday'
    END AS start_dayofweek
FROM bikeshare_trips
"""
df_dayofweek = pd.read_sql(dayofweek_query, conn)
df_dayofweek

,start_time,start_dayofweek
0,2015-03-19 19:12:00,Thursday
1,2016-10-30 02:06:04,Sunday
2,2016-03-11 16:28:27,Friday
3,2014-11-23 15:12:00,Sunday
4,2017-04-16 15:39:13,Sunday
...,...,...
649226,2014-07-06 00:12:00,Sunday
649227,2017-02-06 17:20:44,Monday
649228,2015-11-18 13:12:55,Wednesday
649229,2016-05-29 14:45:34,Sunday


## 2️⃣ จำนวนทริปที่เริ่มต้นจากแต่ละสถานี

In [40]:
station_trip_count_query = """
SELECT start_station_id, COUNT(*) AS trip_count
FROM bikeshare_trips
GROUP BY start_station_id
ORDER BY trip_count DESC
LIMIT 10
"""
df_station_trip_count = pd.read_sql(station_trip_count_query, conn)

## 3️⃣ จำนวนทริปที่ใช้เวลามากกว่า 1 ชั่วโมง

In [41]:
long_trip_query = """
SELECT COUNT(*) AS long_trip_count
FROM bikeshare_trips
WHERE duration_minutes > 60
"""
df_long_trips = pd.read_sql(long_trip_query, conn)
df_long_trips

,long_trip_count
0,61079


## 4️⃣ หาชื่อสถานีปลายทางจาก end_station_id

In [42]:
station_name_query = """
SELECT t.end_station_id, s.name AS end_station_name
FROM bikeshare_trips AS t
INNER JOIN bikeshare_stations AS s ON t.end_station_id = s.station_id
GROUP BY t.end_station_id, s.name
ORDER BY end_station_name
"""
df_end_station_info = pd.read_sql(station_name_query, conn)
df_end_station_info

,end_station_id,end_station_name
0,3291.0,11th & San Jacinto
1,3635.0,13th & San Antonio
2,2540.0,17th & Guadalupe
3,2494.0,2nd & Congress
4,2552.0,3rd & West
...,...,...
66,2548.0,UT West Mall @ Guadalupe
67,2536.0,Waller & 6th St.
68,2537.0,West & 6th St.
69,2574.0,Zilker Park


## 5️⃣ หาสถานีที่จักรยานหมายเลข 400 ถูกขับออกจากบ่อยที่สุด

In [43]:
bike_400_query = """
SELECT start_station_id, COUNT(*) AS bike_400_trip_count
FROM bikeshare_trips
WHERE bikeid = 400
GROUP BY start_station_id
ORDER BY bike_400_trip_count DESC
"""
df_bike_400_trips = pd.read_sql(bike_400_query, conn)
df_bike_400_trips

,start_station_id,bike_400_trip_count
0,2499.0,74
1,2494.0,72
2,2495.0,67
3,2501.0,62
4,2707.0,57
...,...,...
60,1002.0,2
61,3687.0,1
62,3464.0,1
63,3381.0,1


# 🔥 ตั้งโจทย์ใหม่ 5 ข้อ 🔥

## 1️⃣ หาวันที่มีการใช้งานจักรยานสูงสุด

In [44]:
most_active_day_query = """
SELECT DATE(start_time) AS trip_date, COUNT(*) AS trip_count
FROM bikeshare_trips
GROUP BY trip_date
ORDER BY trip_count DESC
LIMIT 1
"""
df_most_active_day = pd.read_sql(most_active_day_query, conn)
df_most_active_day

,trip_date,trip_count
0,2015-03-19,3031


## 2️⃣ หา bikeid ที่ถูกใช้บ่อยที่สุด

In [45]:
most_used_bike_query = """
SELECT bikeid, COUNT(*) AS usage_count
FROM bikeshare_trips
GROUP BY bikeid
ORDER BY usage_count DESC
LIMIT 1
"""
df_most_used_bike = pd.read_sql(most_used_bike_query, conn)
df_most_used_bike

,bikeid,usage_count
0,951.0,2049


## 3️⃣ คำนวณระยะเวลาทริปเฉลี่ยในแต่ละวัน


In [46]:
avg_trip_duration_query = """
SELECT STRFTIME('%w', start_time) AS day_of_week, AVG(duration_minutes) AS avg_duration
FROM bikeshare_trips
GROUP BY day_of_week
ORDER BY avg_duration DESC
"""
df_avg_trip_duration = pd.read_sql(avg_trip_duration_query, conn)
df_avg_trip_duration

,day_of_week,avg_duration
0,6,35.321557
1,0,33.975284
2,5,28.992915
3,1,26.022160
4,4,24.790738
5,2,23.300893
6,3,22.777779


## 4️⃣ หาจำนวนครั้งที่จักรยานแต่ละคันถูกใช้งานเกิน 3 ชั่วโมง


In [47]:
bike_long_trip_query = """
SELECT bikeid, COUNT(*) AS long_trip_count
FROM bikeshare_trips
WHERE duration_minutes > 180
GROUP BY bikeid
ORDER BY long_trip_count DESC
LIMIT 5
"""
df_bike_long_trips = pd.read_sql(bike_long_trip_query, conn)
df_bike_long_trips

,bikeid,long_trip_count
0,52.0,44
1,999.0,42
2,897.0,42
3,866.0,42
4,135.0,41


## 5️⃣ หาสถานีที่มีจำนวนทริปไปจอดมากที่สุด


In [48]:
most_arrived_station_query = """
SELECT end_station_id, COUNT(*) AS arrival_count
FROM bikeshare_trips
GROUP BY end_station_id
ORDER BY arrival_count DESC
LIMIT 10
"""
df_most_arrived_station = pd.read_sql(most_arrived_station_query, conn)
df_most_arrived_station

,end_station_id,arrival_count
0,2499.0,33125
1,2494.0,29516
2,2575.0,28023
3,2495.0,27902
4,2498.0,26862
5,2501.0,25070
6,2707.0,22696
7,2563.0,21916
8,NaN,19842
9,2566.0,18632


In [49]:
# ✅ ปิดการเชื่อมต่อฐานข้อมูล
conn.close()